In [15]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00257/Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.xls

--2018-08-01 17:04:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/00257/Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.xls
Resolving archive.ics.uci.edu... 128.195.10.249
Connecting to archive.ics.uci.edu|128.195.10.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57856 (56K) [application/vnd.ms-excel]
Saving to: 'Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls'

Data_User_Modeling_ 100%[===================>]  56.50K   135KB/s    in 0.4s    

2018-08-01 17:04:37 (135 KB/s) - 'Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls' saved [57856/57856]



In [1]:
import pandas as pd
data_train = pd.read_excel("Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls", sheet_name=1, usecols=5)
data_test = pd.read_excel("Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.xls", sheet_name=2, usecols=5)
data_test = data_test.replace("Very Low", "very_low")
print("Number of samples: %d" % len(data_train))
data_train.head()

Number of samples: 258


,STG,SCG,STR,LPR,PEG,UNS
0,0.00,0.00,0.00,0.00,0.00,very_low
1,0.08,0.08,0.10,0.24,0.90,High
2,0.06,0.06,0.05,0.25,0.33,Low
3,0.10,0.10,0.15,0.65,0.30,Middle
4,0.08,0.08,0.08,0.98,0.24,Low


In [2]:
data_train.describe(include='all')

,STG,SCG,STR,LPR,PEG,UNS
count,258.000000,258.000000,258.000000,258.000000,258.000000,258
unique,NaN,NaN,NaN,NaN,NaN,4
top,NaN,NaN,NaN,NaN,NaN,Middle
freq,NaN,NaN,NaN,NaN,NaN,88
mean,0.371147,0.355674,0.468004,0.432713,0.458539,NaN
std,0.210271,0.211962,0.245940,0.248108,0.255211,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.240750,0.210000,0.291250,0.250000,0.250000,NaN
50%,0.327000,0.302500,0.490000,0.330000,0.500000,NaN
75%,0.495000,0.497500,0.690000,0.647500,0.660000,NaN


In [3]:
import numpy as np
import sklearn
from sklearn import model_selection, tree

train_y = data_train[" UNS"].values
test_y = data_test[" UNS"].values
y = np.concatenate((train_y, test_y))
data_train.drop(" UNS", axis=1, inplace=True)
data_test.drop(" UNS", axis=1, inplace=True)
train_X = data_train.values.astype(np.float)
test_X = data_test.values.astype(np.float)

In [4]:
model = tree.DecisionTreeClassifier()
model = model.fit(train_X, train_y)

In [5]:
model.score(test_X, test_y)

0.8827586206896552

In [6]:
params = {
    "min_samples_leaf": list(range(5, 30, 1)),
    "min_samples_split": list(range(2, 4, 1)),
    "criterion": ["gini", "entropy"],
}
gs = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), params, n_jobs=3, verbose=1)
gs = gs.fit(train_X, train_y)
gs.score(test_X, test_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=3)]: Done 300 out of 300 | elapsed:    0.4s finished


0.9241379310344827

In [7]:
gs.best_params_

{'criterion': 'entropy', 'min_samples_leaf': 5, 'min_samples_split': 2}

In [8]:
model = tree.DecisionTreeClassifier(min_samples_leaf=5, min_samples_split=2, criterion="entropy")
model = model.fit(train_X, train_y)
print(model.score(train_X, train_y), model.score(test_X, test_y))

0.9612403100775194 0.9241379310344827


In [9]:
from sklearn.tree import _tree

total = 0

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            global total
            total += tree_.value[node].sum()
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

#tree_to_code(model, data_train.columns)

## Reduced error pruning

In [10]:
from copy import deepcopy
from sklearn.tree._tree import TREE_LEAF

In [11]:
def tree_depth(tree):
    max_depth = 0
    stack = [(0,0)]
    while stack:
        node, depth = stack.pop()
        if depth > max_depth:
            max_depth = depth
        if tree.children_left[node] != TREE_LEAF:
            stack.append((tree.children_left[node], depth+1))
        if tree.children_right[node] != TREE_LEAF:
            stack.append((tree.children_right[node], depth+1))
    return max_depth

In [12]:
def get_parents(tree):
    parents = {x:i for i, x in enumerate(tree.children_left) if x != TREE_LEAF}
    parents.update({x:i for i, x in enumerate(tree.children_right) if x != TREE_LEAF})
    return parents

def prune(model, validation_X, validation_y, possible_score_drop=-0.02, verbose=False):
    model = deepcopy(model)
    tree = model.tree_
    curent_score = model.score(validation_X, validation_y)
    changes = True
    while changes:
        changes = False
        leafs = np.where(tree.children_left == TREE_LEAF)[0]
        parents = get_parents(tree)
        for leaf in leafs:
            if leaf not in parents:
                continue
            parent = parents[leaf]
            _child_left = model.tree_.children_left[parent]
            _child_right = model.tree_.children_right[parent]
            _feature = model.tree_.feature[parent]
            model.tree_.children_left[parent] = TREE_LEAF
            model.tree_.children_right[parent] = TREE_LEAF
            model.tree_.feature[parent] = _tree.TREE_UNDEFINED
            new_score = model.score(validation_X, validation_y)
            #print(curent_score, new_score, (new_score - curent_score) / curent_score)
            score_change = (new_score - curent_score) / curent_score
            if score_change < possible_score_drop:
                # Bad cut
                model.tree_.children_left[parent] = _child_left
                model.tree_.children_right[parent] = _child_right
                model.tree_.feature[parent] = _feature
            else:
                if verbose:
                    print("Remove {:2} and {:2} leafs. Score change: {:.3f}".format(_child_right, _child_left, score_change))
                changes = True
                break
    return model

In [13]:
pruned_model = prune(model, test_X, test_y, verbose=True)
hard_pruned_model = prune(model, test_X, test_y, verbose=True, possible_score_drop=-0.05)

Remove  4 and  3 leafs. Score change: 0.000
Remove  6 and  5 leafs. Score change: 0.000
Remove 10 and  9 leafs. Score change: 0.000
Remove 15 and 14 leafs. Score change: 0.007
Remove 20 and 19 leafs. Score change: 0.007
Remove 24 and 23 leafs. Score change: 0.007
Remove  4 and  3 leafs. Score change: 0.000
Remove  6 and  5 leafs. Score change: 0.000
Remove 10 and  9 leafs. Score change: 0.000
Remove 11 and  8 leafs. Score change: -0.030
Remove 13 and 12 leafs. Score change: -0.030
Remove 15 and 14 leafs. Score change: -0.030
Remove 20 and 19 leafs. Score change: -0.030
Remove 24 and 23 leafs. Score change: -0.030


In [14]:
def tree_to_rules(tree, feature_names):
    from sklearn.tree import _tree as Tree
    
    tree = tree.tree_
    feature_names = [
        feature_names[i] if i != Tree.TREE_UNDEFINED else "TREE_UNDEFINED"
        for i in tree.feature
    ]
    queue = [(0, tuple())]
    rules = []
    while queue:
        node, path = queue.pop()
        if tree.feature[node] != Tree.TREE_UNDEFINED:
            name = feature_names[node]
            threshold = tree.threshold[node]
            queue.append((tree.children_left[node], path + ((name, False, threshold),)))  # <= threshold
            queue.append((tree.children_right[node], path + ((name, True, threshold),)))  # > threshold
        else:
            rules.append((path, tuple(tree.value[node][0])))
    return rules

In [15]:
rules = tree_to_rules(model, data_train.columns)
rules_pruned = tree_to_rules(pruned_model, data_train.columns)
rules_hard_pruned = tree_to_rules(hard_pruned_model, data_train.columns)

In [16]:
rules_hard_pruned

[((('PEG', True, 0.375), ('PEG', True, 0.675000011920929)),
  (52.0, 0.0, 1.0, 0.0)),
 ((('PEG', True, 0.375),
   ('PEG', False, 0.675000011920929),
   ('LPR', True, 0.8300000429153442)),
  (10.0, 0.0, 0.0, 0.0)),
 ((('PEG', True, 0.375),
   ('PEG', False, 0.675000011920929),
   ('LPR', False, 0.8300000429153442)),
  (1.0, 0.0, 74.0, 0.0)),
 ((('PEG', False, 0.375), ('PEG', True, 0.13499999046325684)),
  (0.0, 74.0, 13.0, 3.0)),
 ((('PEG', False, 0.375), ('PEG', False, 0.13499999046325684)),
  (0.0, 9.0, 0.0, 21.0))]

In [17]:
(sum([len(r[0]) for r in rules]), 
 sum([len(r[0]) for r in rules_pruned]), 
 sum([len(r[0]) for r in rules_hard_pruned]))

(50, 21, 12)

In [18]:
print(len(rules), tree_depth(model.tree_))
print(len(rules_pruned), tree_depth(pruned_model.tree_))
print(len(rules_hard_pruned), tree_depth(hard_pruned_model.tree_))

13 5
7 4
5 3


In [19]:
hard_pruned_model.score(test_X, test_y)

0.896551724137931

In [20]:
pruned_model.score(test_X, test_y)

0.9310344827586207

In [21]:
model.score(test_X, test_y)

0.9241379310344827